In [47]:
import os
import pprint
import absl
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()
import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
# TODO: 나중에 확인
# from tfx.components import ResolverNode  
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
# TODO: 나중에 확인
# from tfx.utils.dsl_utils import external_input

from google.protobuf import text_format
from tensorflow_data_validation.utils import io_util
from tensorflow_metadata.proto.v0 import anomalies_pb2

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

The tfx.orchestration.experimental.interactive.notebook_extensions.skip extension is already loaded. To reload it, use:
  %reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip


In [2]:
_DATA_ROOT = './data'
_DATA_FILEPATH = os.path.join(_DATA_ROOT, "advertising.csv")

_PIPELINE_ROOT = './pipeline'
_PIPELINE_NAME = 'advert-test'
_META_DATA_PATH = os.path.join(_PIPELINE_ROOT, "metadata.db")

## Helper functions

In [9]:
# ExampleGen, Transformation 결과 확인
def load_sample_tfrecord_from_uri(uri, sample_number, compression_type="GZIP"):
    tfrecord_filenames = [os.path.join(uri, name)
                         for name in os.listdir(uri)]
    dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type)
    
    for tfrecord in dataset.take(1):
        serialized_example = tfrecord.numpy()
        example = tf.train.Example()
        example.ParseFromString(serialized_example)
        print(example)

In [60]:
# StatisticsGen
def load_stats_from_uri(uri):
    file_name = os.listdir(uri)[0]
    file_format = file_name.split('.')[1]
    file_path = os.path.join(uri, file_name)
    
    if file_format == 'pb':
        result = tfdv.load_stats_binary(file_path)
    elif file_format == 'pbtxt':
        result = tfdv.load_stats_text(file_path)
    else:
        result = None
    
    return result


# SchemaGen
def load_schema_from_uri(uri):
    file_name = os.listdir(uri)[0]
    file_path = os.path.join(uri, file_name)
    
    result = tfdv.load_schema_text(file_path)
    
    return result


# ExampleValidator
def load_anomalies_from_uri(uri):
    result = None
    file_name = os.listdir(uri)[0]
    file_format = file_name.split('.')[1]
    file_path = os.path.join(uri, file_name)
    
    if file_format == 'pb':
        # Source code
        anomalies_proto = anomalies_pb2.Anomalies()
        anomalies_proto.ParseFromString(io_util.read_file_to_string(
          file_path, binary_mode=True))  # binary mode true로 해서 parsing
        result = anomalies_proto
        
    elif file_format == 'pbtxt':
        result = tfdv.load_anomalies_binary(file_path)
        # Source code
#         anomalies = anomalies_pb2.Anomalies()
#         anomalies_text = io_util.read_file_to_string(input_path)
#         text_format.Parse(anomalies_text, anomalies)
#         result = anomalies
        
    return result

## Interactive Context 환경 생성

In [3]:
# Interactive context에서는 로컬에 지정된 pipline_root로 artifact들이 생성됨
# Production pipeline 구성 시에는 google storage, S3 같은 object storage에 저장
context = InteractiveContext(pipeline_root=_PIPELINE_ROOT,
                             pipeline_name=_PIPELINE_NAME,
                             metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(_META_DATA_PATH))

## TFDV

### ExampleGen

In [4]:
example_gen = CsvExampleGen(input_base=_DATA_ROOT)

In [5]:
context.run(example_gen)  # run 해야 생성됨

/Users/jinwoohong/.pyenv/versions/tfx/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [6]:
# artifcat 자체 접근
artifact = example_gen.outputs['examples'].get()[0]  

In [7]:
print(artifact.split_names, artifact.uri)

["train", "eval"] ./pipeline/CsvExampleGen/examples/1


In [8]:
artifact

.type,<class 'tfx.types.standard_artifacts.Examples'>
.uri,./pipeline/CsvExampleGen/examples/1
.span,0
.split_names,"[""train"", ""eval""]"
.version,0


In [10]:
# 샘플 예시 보기
# Get the URI of the output artifact representing the transformed examples
raw_train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

load_sample_tfrecord_from_uri(raw_train_uri, 1)

features {
  feature {
    key: "AdTopicLine"
    value {
      bytes_list {
        value: "Cloned 5thgeneration orchestration"
      }
    }
  }
  feature {
    key: "Age"
    value {
      int64_list {
        value: 35
      }
    }
  }
  feature {
    key: "AreaIncome"
    value {
      float_list {
        value: 61833.8984375
      }
    }
  }
  feature {
    key: "City"
    value {
      bytes_list {
        value: "Wrightburgh"
      }
    }
  }
  feature {
    key: "ClickedOnAd"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Country"
    value {
      bytes_list {
        value: "Tunisia"
      }
    }
  }
  feature {
    key: "DailyInternetUsage"
    value {
      float_list {
        value: 256.0899963378906
      }
    }
  }
  feature {
    key: "DailyTimeSpentOnSite"
    value {
      float_list {
        value: 68.94999694824219
      }
    }
  }
  feature {
    key: "Male"
    value {
      int64_list {
        value: 0
      }
 

2022-04-15 23:34:10.947590: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### StatisticsGen

In [11]:
statistics_gen = StatisticsGen(
   examples=example_gen.outputs['examples'])

In [12]:
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [64]:
# 아래 코드와 같음
# statistics_gen.outputs['statistics']._artifacts
# statistics_gen.outputs['statistics'].get()
stats_uri = statistics_gen.outputs['statistics'].get()[0].uri
stats_uri

'./pipeline/StatisticsGen/statistics/2'

In [67]:
# 값 접근 가능
# load_stats_from_uri(os.path.join(stats_uri, 'Split-eval'))

In [14]:
statistics_gen.outputs['statistics']

OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={})

In [63]:
context.show(statistics_gen.outputs['statistics'])

### SchemaGen

In [23]:
schema_gen = SchemaGen(
   statistics=statistics_gen.outputs['statistics'],
   infer_feature_shape=False)  # False로 해야 나중에 missing 값 나올 때 조정해도 shape anomalies 안 나옴. 꼭 지정해야 하는 shape는 나중에 따로 지정

In [24]:
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [68]:
schema_uri = schema_gen.outputs['schema'].get()[0].uri
schema_uri

'./pipeline/SchemaGen/schema/3'

In [72]:
# 값 접근 가능
# load_schema_from_uri(schema_uri)

In [25]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'AdTopicLine',BYTES,required,single,-
'Age',INT,required,single,-
'AreaIncome',FLOAT,required,single,-
'City',BYTES,required,single,-
'ClickedOnAd',INT,required,single,-
'Country',BYTES,required,single,-
'DailyInternetUsage',FLOAT,required,single,-
'DailyTimeSpentOnSite',FLOAT,required,single,-
'Male',INT,required,single,-


### ExampleValidator

In [73]:
example_validator = ExampleValidator(
   statistics=statistics_gen.outputs['statistics'],
   schema=schema_gen.outputs['schema'])

In [74]:
example_validator

ExampleValidator(spec: <tfx.types.standard_component_specs.ExampleValidatorSpec object at 0x1754e9b50>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x1754e98b0>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: ExampleValidator, inputs: {'statistics': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}), 'schema': OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={})}, outputs: {'anomalies': OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={})})

In [75]:
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 5
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [80]:
example_validator_uri = example_validator.outputs['anomalies'].get()[0].uri
example_validator_uri

'./pipeline/ExampleValidator/anomalies/4'

In [79]:
# 값 확인 가능
# load_anomalies_from_uri(os.path.join(example_validator_uri, 'Split-eval'))

In [18]:
context.show(example_validator.outputs['anomalies'])

In [33]:
example_validator.outputs['anomalies']

OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={})

## Transform

In [19]:
advert_transform = 'model/advert_transform.py'
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=advert_transform)

In [20]:
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying advert_transform.py -> build/lib
copying advert_constants.py -> build/lib
installing to /var/folders/4b/sf_49brs005d5sy8ql2r7yq80000gn/T/tmpz3dx2kf9
running install
running install_lib
copying build/lib/advert_transform.py -> /var/folders/4b/sf_49brs005d5sy8ql2r7yq80000gn/T/tmpz3dx2kf9
copying build/lib/advert_constants.py -> /var/folders/4b/sf_49brs005d5sy8ql2r7yq80000gn/T/tmpz3dx2kf9
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code

2022-04-15 15:29:46.315516: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-15 15:29:53.868636: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./pipeline/Transform/transform_graph/15/.temp_path/tftransform_tmp/f1ba798704e04bcd81c042a5a0b44bc4/assets
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:Assets written to: ./pipeline/Transform/transform_graph/15/.temp_path/tftransform_tmp/00dadd1e5bb64b058989127f8c7e31f9/assets
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 15
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [32]:
transform.outputs['transformed_examples'].get()[0]

.type,<class 'tfx.types.standard_artifacts.Examples'>
.uri,./pipeline/Transform/transformed_examples/15
.span,0
.split_names,"[""train"", ""eval""]"
.version,0


In [36]:
# 샘플 예시 보기
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Decode the first record and print output
for tfrecord in dataset.take(1):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

features {
  feature {
    key: "Age_xf"
    value {
      float_list {
        value: -0.08752329647541046
      }
    }
  }
  feature {
    key: "AreaIncome_xf"
    value {
      float_list {
        value: 0.5186453461647034
      }
    }
  }
  feature {
    key: "City_xf"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "ClickedOnAd_xf"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Country_xf"
    value {
      int64_list {
        value: 133
      }
    }
  }
  feature {
    key: "DailyInternetUsage_xf"
    value {
      float_list {
        value: 1.7440475225448608
      }
    }
  }
  feature {
    key: "DailyTimeSpentOnSite_xf"
    value {
      float_list {
        value: 0.2472444623708725
      }
    }
  }
  feature {
    key: "Male_xf"
    value {
      int64_list {
        value: 0
      }
    }
  }
}



In [37]:
# Get URI and list subdirectories
# The transformed_metadata subdirectory contains the schema of the preprocessed data.
# The transform_fn subdirectory contains the actual preprocessing graph.
# The metadata subdirectory contains the schema of the original data.
graph_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(graph_uri)

['transformed_metadata', 'transform_fn', 'metadata']